In [1]:
from rlrom.trainers import RLTrainer
from rlrom.testers import RLTester
import os, sys, glob
from rlrom.utils import *
from pprint import pprint
import polars as pl
from datetime import date

2025-10-14 21:44:30.100492: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 21:44:30.133203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-14 21:44:30.675782: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
cfg_file = 'cartpole/cfg0tr_ppo_specs.yml'
#cfg_file = 'highway_env/cfg0tr.yml'
#T = RLTrainer(cfg_file)
cfg = load_cfg(cfg_file)


loading field [ specs ] from STL file [ cartpole.stl ]
loading field [ ppo ] from YAML file [ cartpole_ppo.yml ]


In [3]:
df = get_df_all_trainings(cfg)
print(df.collect())

shape: (1, 4)
┌────────────┬──────────┬─────────────────────────────────┬─────────────────────────────────┐
│ date       ┆ training ┆ checkpoints                     ┆ path                            │
│ ---        ┆ ---      ┆ ---                             ┆ ---                             │
│ date       ┆ str      ┆ object                          ┆ str                             │
╞════════════╪══════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 2025-10-02 ┆ 0        ┆ naive plan: (run LazyFrame.exp… ┆ /home/alex/workspace/rlrom/exa… │
└────────────┴──────────┴─────────────────────────────────┴─────────────────────────────────┘


In [ ]:
# get last training
df_training = get_df_training_res(df.collect()['checkpoints'][0])
print(df_training.collect().select(pl.exclude('')))


shape: (20, 9)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ steps  ┆ res_files  ┆ model_file ┆ res       ┆ … ┆ mean_ep_r ┆ cart_on_l ┆ phi_left_ ┆ reward_fo │
│ ---    ┆ ---        ┆ s          ┆ ---       ┆   ┆ ew        ┆ eft       ┆ goal      ┆ rmulas    │
│ i64    ┆ str        ┆ ---        ┆ struct[4] ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆            ┆ str        ┆           ┆   ┆ f64       ┆ struct[3] ┆ struct[2] ┆ struct[1] │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 10000  ┆ /home/alex ┆ /home/alex ┆ {{[0.0,   ┆ … ┆ 27.5      ┆ {-0.25013 ┆ {-0.47302 ┆ {{0.0,-0. │
│        ┆ /workspace ┆ /workspace ┆ -0.428293 ┆   ┆           ┆ 1,0.0,-6. ┆ 3,0.0}    ┆ 250131,-6 │
│        ┆ /rlrom/exa ┆ /rlrom/exa ┆ , … -0.49 ┆   ┆           ┆ 703729}   ┆           ┆ .703729}} │
│        ┆ …          ┆ …          ┆ 7704…     ┆   ┆           ┆           ┆

shape: (8, 9)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ steps  ┆ res_files  ┆ model_file ┆ ep_len    ┆ … ┆ mean_ep_l ┆ mean_ep_r ┆ eval_form ┆ reward_fo │
│ ---    ┆ ---        ┆ s          ┆ ---       ┆   ┆ en        ┆ ew        ┆ ulas      ┆ rmulas    │
│ i64    ┆ str        ┆ ---        ┆ list[i64] ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆            ┆ str        ┆           ┆   ┆ f64       ┆ f64       ┆ struct[0] ┆ struct[0] │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 15000  ┆ /home/alex ┆ /home/alex ┆ [28, 23,  ┆ … ┆ 41.5      ┆ 41.5      ┆ {}        ┆ {}        │
│        ┆ /workspace ┆ /workspace ┆ … 22]     ┆   ┆           ┆           ┆           ┆           │
│        ┆ /rlrom/exa ┆ /rlrom/exa ┆           ┆   ┆           ┆           ┆           ┆           │
│        ┆ …          ┆ …          ┆           ┆   ┆           ┆           ┆ 

In [ ]:
# get steps
steps = df_cp.collect()['steps'][:]
steps = steps.to_numpy()
print(steps)

In [8]:
cfg_specs= cfg.get('cfg_specs')

In [9]:
pprint(cfg_specs)

{'action_names': {'push': 'action'},
 'eval_formulas': {'cart_on_left': {'eval_all_steps': True, 'past_horizon': 0},
                   'phi_left_goal': {'eval_all_steps': False, 't0': 0}},
 'obs_names': {'theta': 'obs[2]',
               'theta_dot': 'obs[3]',
               'x': 'obs[0]',
               'x_dot': 'obs[1]'},
 'reward_formulas': {'cart_on_left': {'lower_bound': 0,
                                      'past_horizon': 0,
                                      'weight': 10}},
 'specs': 'signal push,x,x_dot,theta,theta_dot,reward\n'
          '\n'
          'param epsi= 0.03\n'
          'pole_up := abs(theta[t]) < epsi\n'
          'keep_up := alw_[0,100] pole_up\n'
          'phi_goal_keep_up := ev_[0, 400] keep_up\n'
          '\n'
          'go_left := x_dot[t] < 0\n'
          'cart_on_left := x[t] < -0.5 and x[t]> -0.7\n'
          'phi_left_goal := ev_[0,400] alw_[0, 100] cart_on_left\n'}


In [ ]:

if cfg_specs is not None:
    eval_phis = cfg_specs.get('eval_formulas', [])
    for f in eval_phis:
        print(f)


cart_on_left
phi_left_goal
